In [1]:
'''
this script loads the *.json files for skills of each market and maps the excel with the traceablity to each of the set of skills 
It also filters repetitions and skills without name if any
'''

'\nthis script loads the *.json files for skills of each market and maps the excel with the traceablity to each of the set of skills \nIt also filters repetitions and skills without name if any\n'

In [2]:
#find all file.json files 

import os
def getFiles(dirName):
    listOfFile = os.listdir(dirName)
    completeFileList = list()
    for file in listOfFile:
        completePath = os.path.join(dirName, file)
        if os.path.isdir(completePath):
            completeFileList = completeFileList + getFiles(completePath)
        elif 'file.json' in completePath:
            completeFileList.append(completePath.replace('\\', '/'))

    return completeFileList

allf = getFiles( "skills/")
print(allf)

['skills/US/file.json', 'skills/IN/file.json', 'skills/IT/file.json', 'skills/CA/file.json', 'skills/UK/file.json', 'skills/AU/file.json', 'skills/MX/file.json', 'skills/DE/file.json', 'skills/FR/file.json', 'skills/ES/file.json', 'skills/JP/file.json']


In [3]:
#load all objects in the json files in a dictoinary with key = filepath
import json
def loadObjects(jsonPath):
    lines = open(jsonPath, 'r').readlines()
    allobj = []
    for line in lines: 
        if(len(line.strip()) > 0):
            try:
                allobj.append(json.loads(line))
            except:
                pass
    return allobj

#NOTICE allo might contain repetitoins! (we filter them later)
allo = {}
for f in allf:
    allo[f] = loadObjects(f)
#list all markets
for key,value in allo.items():
    print(key)
    print(len(value))

skills/US/file.json
90281
skills/IN/file.json
35467
skills/IT/file.json
5271
skills/CA/file.json
42429
skills/UK/file.json
38707
skills/AU/file.json
37071
skills/MX/file.json
2308
skills/DE/file.json
31880
skills/FR/file.json
2575
skills/ES/file.json
17623
skills/JP/file.json
16054


### Load and clean the csv containing the traceability analysis

In [4]:
# This combine manual traceability file from NDSS with SKillvet manual Traceability
# import pandas as pd
# #Read csv in pandas
# dfold = pd.read_csv("Traceability.csv", header=0, encoding='utf-8')

# header_lists = ['Skills', 'Developer', 'Permission', 'Traceability', 'Category',
#        'sub-category', 'Account Linking', 'customer reviews', 'Rating',
#        'Cost',"d","y"]
# dfnew = pd.read_csv('ManualTraceability/data/Manual_Traceability3_15.10_21.46.csv', header=1,encoding='utf-8', names= header_lists)  

# dfnew = dfnew.drop(['d','y'], axis=1)
# dfnew.head(2)
# dfold.head(3)

# for index, row in dfnew.iterrows():
#     for index2, row2 in dfold.iterrows():
#         if row['Skills']==row2['Skills']:
#             dfnew.iloc[index, 4] =  dfold.iloc[index2, 4]
#             dfnew.iloc[index, 5] =  dfold.iloc[index2, 5]
#             dfnew.iloc[index, 6] =  dfold.iloc[index2, 6]
#             dfnew.iloc[index, 7] =  dfold.iloc[index2, 7]
#             dfnew.iloc[index, 8] =  dfold.iloc[index2, 8]
#             dfnew.iloc[index, 9] =  dfold.iloc[index2, 9]
#             break
# dfnew.head(10)

# dfnew.to_csv('Manual_Traceability.csv')

In [5]:
import pandas as pd

#Read Manual Traceability csv in pandas
df = pd.read_csv('Traceability/data/Manual_Traceability.csv',header=0, encoding='utf-8')  
df.head(n=10)
# df.columns

,Skills,Developer,Permission,Traceability,Category,sub-category,Account Linking,customer reviews,Rating,Cost
0,Adrixus,by Adrixus,"['Full Name', 'Email Address', 'Mobile Number']",b,Business & Finance,Business & Finance,NaN,No customer reviews,NaN,Free to Enable
1,Alice,by Solinftec,['Email Address'],b,Business & Finance,Business & Finance,NaN,No customer reviews,NaN,Free to Enable
2,AMERICAN LABOR SERVICES,by That's Easier,"['Email Address', 'Full Name']",b,Business & Finance,Business & Finance,Account linking required,No customer reviews,NaN,Free to Enable
3,AppViewX Certificate Automation,by AppVIewx,"['Lists Read Access', 'Lists Write Access']",b,Business & Finance,Business & Finance,NaN,NaN,5.0 out of 5 stars,Free to Enable
4,Appviewx dashboard creation,by AppVIewx,"['Lists Read Access', 'Lists Write Access', 'D...",b,Business & Finance,Business & Finance,NaN,No customer reviews,1.0 out of 5 stars,Free to Enable
5,Athenia Insure,by Default User,"['Device Address', 'Full Name', 'Email Address...",b,Business & Finance,Business & Finance,NaN,No customer reviews,NaN,Free to Enable
6,Bajaj Allianz,by Yellow Messenger,['Device Address'],b,Business & Finance,Business & Finance,NaN,No customer reviews,5.0 out of 5 stars,Free to Enable
7,cognito form RC1,by CognitoFormsDev Team,['Device Address'],b,Business & Finance,Business & Finance,NaN,NaN,3.9 out of 5 stars,NaN
8,Connexus,by Connexus Credit Union,['Device Address'],b,Business & Finance,Business & Finance,NaN,NaN,3.7 out of 5 stars,NaN
9,Crypto Price,by craig.langford,['Device Country and Postal Code'],b,Business & Finance,Business & Finance,NaN,No customer reviews,NaN,Free to Enable


### Map the traceability to each skill

In [6]:
#filter repetitoins and skills without name or developer
import json, re
def _createSkillDict(oldd):
    '''
    example json obj
    {'Main Category': 'Business & Finance', 'SubCategory': 'Business & Finance', 'Sample_Invocation_Utterances': ["“'Alexa, Open Current Bitcoin'”", "“'Alexa, Ask Current Bitcoin for Prices'”", "“'Alexa, Ask Current Bitcoin what the price is'”"], 'Name': 'CurrentBitcoin', 'Developer': 'by Sarim Studios', 'Skill_permission': [], 'Account_linking': '', 'Review_Count': None, 'Rating': '', 'Total_customer_that_rate_the_skill': '0', 'Total_Customers_Reviews': 'No customer reviews', 'Cost': 'Free to Enable', 'In_skill_purchase': None, 'Skill_description': "Description\nAn 'easy to use' Alexa skill that will provide you with the latest price for a bitcoin. This skill uses BitStamp's latest prices, which are always up-to-date. All you have to do to begin using the skill is ask Alexa, Ask Current Bitcoin for Prices. Price is given in USD.", 'Skill_link': 'https://www.amazon.ca/Sarim-Studios-CurrentBitcoin/dp/B01N9SS2LI/ref=sr_1_635?dchild=1&fst=as%3Aoff&qid=1595709096&rnid=16286270011&s=alexa-skills&sr=1-635', 'privacy_policy and Terms_of_use specified': 'No privacy_policy and Terms_of_use specified'}
    '''
    newd = {}    
    for obj in oldd:
        if len(obj) == 0:
            continue
            
        name = ''
        if 'Name' in obj:
            name = obj['Name']
        else:
            continue
            #print('[!] No name found', obj)
            
        skill_id = ''
        if 'Skill_link' in obj:
            try:
                skill_url = obj['Skill_link']
                skill_id = re.search(r"\b[B]+[0-9]+(?:[A-Z0-9]+/)+", skill_url).group().replace('/','')
            except:
                pass
        else:
            continue
            #print('[!] No name found', obj)
            
        dev = ''
        if 'Developer' in obj:
            dev = obj['Developer']
        else:
            continue
            #print('[!] No dev found', obj)
        
        maincat = ''
        if 'Main Category' in obj:
            maincat = obj['Main Category']
            
        subcat = ''
        if 'SubCategory' in obj:
            subcat = obj['SubCategory']
        perm = obj['Skill_permission']
        linking = obj['Account_linking']
        policy = ''
        if 'privacy_policy' in obj:
            policy = obj['privacy_policy']
            
        newd[skill_id] = {'skill_id':skill_id, 'name': name, 'dev':dev, 'perm':perm, 'acc_linking':linking, 'policy':policy ,'cat':maincat, 'subcat':subcat}
        
    return newd

#Create a dictionary of unqiue skills per market
for f in allf:
    print('********************')
    market_name = f[7:9] 
    print(market_name)
    skillsdict = _createSkillDict(allo[f])
    print('#skills', len(skillsdict))
    
    skillsfound = []
    count_skills = 0
    for sk in skillsdict:
        skill  = skillsdict[sk]
        skname = skill['name']
        skdev = skill['dev']
        skill_id = skill['skill_id']
        
                  
        cols = df.loc[ (df['Skills'] == skname) &  (df['Developer'] == skdev),['Permission', 'Traceability',
       'Account Linking', 'customer reviews', 'Rating','Cost']]
        if(len(cols) > 0):
            res = cols.values.tolist()[0]
            skill['permission'] = res[0]
            skill['traceability'] = res[1].strip()
            skill['acc_linking'] = res[2]
            skill['reviews'] = res[3]
            skill['stars'] = res[4]
            skill['free'] = res[5]
            count_skills+=1
            skillsfound.append( skname )
        else:
            skill['traceability'] = 'n'
            skill['reviews'] = None
            skill['stars'] = None
            skill['free'] = None
            
     
    print('[*]', count_skills, 'were identified and updated their traceablity status from the excel file')
    #print('skills found traceablity', skillsfound)
    
    #save analysis in NewFullDataset json
    path = 'NewFullDataset/'+market_name+'.json'
    with open(path, 'w') as json_file:
        print('Saving json in ',path)
        json.dump(skillsdict, json_file)        
            

********************
US
#skills 61362
[*] 1172 were identified and updated their traceablity status from the excel file
Saving json in  NewFullDataset/US.json
********************
IN
#skills 29344
[*] 217 were identified and updated their traceablity status from the excel file
Saving json in  NewFullDataset/IN.json
********************
IT
#skills 4203
[*] 0 were identified and updated their traceablity status from the excel file
Saving json in  NewFullDataset/IT.json
********************
CA
#skills 26027
[*] 230 were identified and updated their traceablity status from the excel file
Saving json in  NewFullDataset/CA.json
********************
UK
#skills 32822
[*] 333 were identified and updated their traceablity status from the excel file
Saving json in  NewFullDataset/UK.json
********************
AU
#skills 23909
[*] 181 were identified and updated their traceablity status from the excel file
Saving json in  NewFullDataset/AU.json
********************
MX
#skills 1972
[*] 0 were identi

# 1 .Load New Full dataset and perform traceablity analysis

In [7]:
#number of developer involved in 
#broken b and unqiue developers with broken policies, 
#complete c -- developer, partial p -- developer, total skills -- total developer

#broken, complete, and partial, and total per main categories 
#broken, complete, and partial, and total per main subcategories

In [8]:
import os
import json

def getFiles(dirName):
    listOfFile = os.listdir(dirName)
    completeFileList = list()
    for file in listOfFile:
        completePath = os.path.join(dirName, file)
        if os.path.isdir(completePath):
            completeFileList = completeFileList + getFiles(completePath)
        elif '.json' in completePath:
            completeFileList.append(completePath.replace('\\', '/'))

    return completeFileList

allf = getFiles( "NewFullDataset/")
print(allf)

#create dictionary of markets and skills
skillsmarketd = {}
for f in allf:
    marketname = f.split('/')[1].split('.json')[0]
    with open(f, 'r') as json_file:
        skillsmarketd[marketname] = json.load(json_file)

print()
print('Total unique skills per market')
for key,value in skillsmarketd.items():
    print(key, ": ", len(value))

['NewFullDataset/DE.json', 'NewFullDataset/UK.json', 'NewFullDataset/MX.json', 'NewFullDataset/US.json', 'NewFullDataset/IT.json', 'NewFullDataset/AU.json', 'NewFullDataset/IN.json', 'NewFullDataset/FR.json', 'NewFullDataset/JP.json', 'NewFullDataset/CA.json', 'NewFullDataset/ES.json']

Total unique skills per market
DE :  9096
UK :  32822
MX :  1972
US :  61362
IT :  4203
AU :  23909
IN :  29344
FR :  2288
JP :  3513
CA :  26027
ES :  4759


### 1.1 Get how many skills have broken, partial and complete policies (over the total), and how many developers participated in each of these categories regardless of the permission

In [9]:
print("### 1.1 Get how many skills have broken, partial and complete policies (over the total), and how many developers participated in each of these categories")
#first join all dictionaries to create a total english market market
#z = {**x, **y}
englishmarkets = ['UK', 'US', 'CA', 'IN', 'AU']
allm = {}
for market in englishmarkets:    
    value = skillsmarketd[market]
    allm = {**allm, **value}
    #print('adding market', market, len(allm))
skillsmarketd['ALLENGLISH']  = allm

for key in englishmarkets + ['ALLENGLISH']:
    print('************')
    print('Market', key)
    value = skillsmarketd[key]
    t = {'b':0, 'p':0, 'c':0, 'd':0, 'n':0, 't':0}  #b, p, c,d,n total policies  -- traceablity policies
    d = {'b':set([]), 'p':set([]), 'c':set([]), 'd':set([]), 'n':set([]), 't':set([])}  #b unique developers, p unique developers, c unique developers, total dev -- traceability unique developers
    
    for skill, skillobj in value.items():
        '''
        skillobj:
        {'name': 'Wake on Lan (WoL)', 'dev': 'by Oscar Penelo', 'perm': [], 'acc_linking': '', 'policy': 'https://www.amazon.com.au/gp/help/customer/display.html?nodeId=201602230', 'cat': 'Smart Home', 'subcat': 'Smart Home', 'traceability': 'd', 'reviews': None, 'stars': None, 'free': None}
        '''
        trac = skillobj['traceability'].lower()
        dev = skillobj['dev']
        t[trac] +=1
        d[trac].add(dev)
        t['t'] += 1 #total policies
        d['t'].add(dev) #set of developers
    
    #print the data
    print('           \t      #Skills\t    #Unique devs')
    print('Broken     \t   {:>10} \t {:>15}'.format(t['b'], len(d['b'])) )
    print('Partial    \t   {:>10} \t {:>15}'.format(t['p'], len(d['p'])) )
    print('Complete   \t   {:>10} \t {:>15}'.format(t['c'], len(d['c'])) )
    print('Dead Link  \t   {:>10} \t {:>15}'.format(t['d'], len(d['d'])) )
    print('No Policy  \t   {:>10} \t {:>15}'.format(t['n'], len(d['n'])) )
    print('Total      \t   {:>10} \t {:>15}'.format(t['t'], len(d['t'])) )

### 1.1 Get how many skills have broken, partial and complete policies (over the total), and how many developers participated in each of these categories
************
Market UK
           	      #Skills	    #Unique devs
Broken     	           68 	              59
Partial    	           61 	              49
Complete   	          204 	             163
Dead Link  	            0 	               0
No Policy  	        32489 	           14326
Total      	        32822 	           14487
************
Market US
           	      #Skills	    #Unique devs
Broken     	          255 	             195
Partial    	          215 	             165
Complete   	          702 	             373
Dead Link  	            0 	               0
No Policy  	        60190 	           26578
Total      	        61362 	           27030
************
Market CA
           	      #Skills	    #Unique devs
Broken     	           40 	              36
Partial    	           40 	              32
Complete   	          150 	     

### 1.1.1 Get how many skills have broken, partial and complete policies (over the total), and how many developers participated in each of these categories only considering skills with more than one permission regardless of whether the skills require privacy policy

In [17]:

olderperm = [('Lists Read Access', 'Personal Information'),
             ('Lists Write Access', 'Personal Information'),
             ('lists write access (2)', 'Personal Information'),
             ('First Name', 'Name'),
             ('Full Name', 'Name'),
             ('Given Name', 'Name'),
             ('Device Country and Postcode', 'device country and postal code'),
             ('phone number','Mobile Number'),
             ('zip', 'device country and postal code'),
             ('address','Device Address'),
             ('location','Location Services'),
             ('birthday', 'Personal Information'),
             ('email','Email Address'),
             ('area code', 'device country and postal code'),
             ('gender', 'Personal Information'),
             ('born','Personal Information'),
             ('zipcode', 'device country and postal code'),
             ('postal code', 'device country and postal code') 
            ]
olderrem = ['alexa notifications', 'skill personisation', 'reminders', 'timer', 'birthday', 'gender']



print("1.1.1 Get how many skills have broken, partial and complete policies (over the total), and how many developers participated in each of these categories only considering skills with more than one permission")
#first join all dictionaries to create a total english market market
#z = {**x, **y}
englishmarkets = ['UK', 'US', 'CA', 'IN', 'AU']
allm = {}
for market in englishmarkets:    
    value = skillsmarketd[market]
    allm = {**allm, **value}
    #print('adding market', market, len(allm))
skillsmarketd['ALLENGLISH']  = allm

for key in englishmarkets+['ALLENGLISH']:
    print('************')
    print('Market', key)
    value = skillsmarketd[key]
    t = {'b':0, 'p':0, 'c':0, 'd':0, 'n':0, 't':0}  #b, p, c,d,n total policies  -- traceablity policies
    d = {'b':set([]), 'p':set([]), 'c':set([]), 'd':set([]), 'n':set([]), 't':set([])}  #b unique developers, p unique developers, c unique developers, total dev -- traceability unique developers
    
    for skill, skillobj in value.items():
        '''
        skillobj:
        {'name': 'Wake on Lan (WoL)', 'dev': 'by Oscar Penelo', 'perm': [], 'acc_linking': '', 'policy': 'https://www.amazon.com.au/gp/help/customer/display.html?nodeId=201602230', 'cat': 'Smart Home', 'subcat': 'Smart Home', 'traceability': 'd', 'reviews': None, 'stars': None, 'free': None}
        '''
        trac = skillobj['traceability'].lower().strip()
        dev = skillobj['dev']
        n_perm = len(skillobj['perm'])
        #only consdier these skills with 1 or more permissions
        if(n_perm < 1):
            continue
        
        for mm in skillobj['perm']:
            for perm in olderperm:
                if mm.lower().strip() == perm[0].lower():
                    mm = perm[1].lower()
                    if mm.lower().strip() in olderrem:
                        continue     
                    t[trac] +=1
                    d[trac].add(dev)
                    t['t'] += 1 #total policies
                    d['t'].add(dev) #set of developers
    
    #print the data
    print('           \t              #Skills\t    #Unique devs')
    print('Broken     \t   {:>10} ({:.2f}%)\t {:>15}'.format(t['b'],t['b']/t['t'], len(d['b'])) )
    print('Partial    \t   {:>10} ({:.2f}%)\t {:>15}'.format(t['p'],t['p']/t['t'], len(d['p'])) )
    print('Complete   \t   {:>10} ({:.2f}%)\t {:>15}'.format(t['c'],t['c']/t['t'], len(d['c'])) )
    print('Dead Link  \t   {:>10} ({:.2f}%)\t {:>15}'.format(t['d'],t['d']/t['t'], len(d['d'])) )
    print('No Policy  \t   {:>10} ({:.2f}%)\t {:>15}'.format(t['n'],t['n']/t['t'], len(d['n'])) )
    print('Total      \t   {:>10} ({:.2f}%)\t {:>15}'.format(t['t'],t['t']/t['t'], len(d['t'])) )

1.1.1 Get how many skills have broken, partial and complete policies (over the total), and how many developers participated in each of these categories only considering skills with more than one permission
************
Market UK
           	              #Skills	    #Unique devs
Broken     	           68 (0.12%)	              59
Partial    	           60 (0.10%)	              49
Complete   	          203 (0.35%)	             162
Dead Link  	            0 (0.00%)	               0
No Policy  	          250 (0.43%)	             152
Total      	          581 (1.00%)	             403
************
Market US
           	              #Skills	    #Unique devs
Broken     	          254 (0.15%)	             195
Partial    	          214 (0.13%)	             164
Complete   	          695 (0.41%)	             370
Dead Link  	            0 (0.00%)	               0
No Policy  	          535 (0.32%)	             279
Total      	         1698 (1.00%)	             949
************
Market CA
           

### 1.2 Get how many skills have broken, partial and complete policies (over the total) per skill category and per market

In [11]:
from collections import Counter
print("### 1.2 Get how many skills have broken, partial and complete policies (over the total) per skill category and per market. skills with one or more permissions")

#first join all dictionaries to create a total english market market
#z = {**x, **y}
englishmarkets = ['UK', 'US', 'CA', 'IN', 'AU']
allm = {}
for market in englishmarkets:    
    value = skillsmarketd[market]
    allm = {**allm, **value}
    #print('adding market', market, len(allm))
skillsmarketd['ALLENGLISH']  = allm

for key in englishmarkets+['ALLENGLISH']:
    print()
    print('************')
    print('Market', key)
    print('************')
    value = skillsmarketd[key]

    t = {'b':[], 'p':[], 'c':[], 'd':[], 'n':[], 't':[]}  #b, p, c,d,n total policies  -- traceablity policies category
    st = {'b':[], 'p':[], 'c':[], 'd':[], 'n':[], 't':[]}  #b, p, c,d,n total policies  -- traceablity policies subcategory

    for skill, skillobj in value.items():
        '''
        skillobj example:
        {'name': 'Wake on Lan (WoL)', 'dev': 'by Oscar Penelo', 'perm': [], 'acc_linking': '', 'policy': 'https://www.amazon.com.au/gp/help/customer/display.html?nodeId=201602230', 'cat': 'Smart Home', 'subcat': 'Smart Home', 'traceability': 'd', 'reviews': None, 'stars': None, 'free': None}
        '''
        trac = skillobj['traceability'].lower().strip()
        cat = skillobj['cat']
        scat = skillobj['subcat']
        n_perm = len(skillobj['perm'])
        #only consdier these skills with 1 or more permissions
        if(n_perm < 1):
            continue
        t[trac].append(cat)
        st[trac].append(scat)
        #add for the total
        t['t'].append(cat) #set of developers
        st['t'].append(scat)

    #print the data for CATEGORIES
    print('**CATEGORIES \t   #Skills')
    print('**Broken     \n   {} '.format(Counter(t['b'])) )
    print('**Partial    \n   {} '.format(Counter(t['p'])) )
    print('**Complete   \n   {} '.format(Counter(t['c'])) )
    print('**Dead Link  \n   {} '.format(Counter(t['d'])) )
    print('**No Policy  \n   {} '.format(Counter(t['n'])) )
    print('**Total      \n   {} '.format(Counter(t['t'])) )
    
    
    print()
    print()
    #print the data for SUBCATEGORIES
    print('**SUBCATEGORIES \t   #Skills')
    print('**Broken     \n   {} '.format(Counter(t['b'])) )
    print('**Partial    \n   {} '.format(Counter(t['p'])) )
    print('**Complete   \n   {} '.format(Counter(t['c'])) )
    print('**Dead Link  \n   {} '.format(Counter(t['d'])) )
    print('**No Policy  \n   {} '.format(Counter(t['n'])) )
    print('**Total      \n   {} '.format(Counter(t['t'])) )
    

### 1.2 Get how many skills have broken, partial and complete policies (over the total) per skill category and per market. skills with one or more permissions

************
Market UK
************
**CATEGORIES 	   #Skills
**Broken     
   Counter({'': 19, 'Games': 6, 'Organizers & Assistants': 6, 'Navigation & Trip Planners': 6, 'Religion & Spirituality': 5, 'Home Services': 4, 'Cooking & Recipes': 3, 'Schools': 3, 'Knowledge & Trivia': 2, 'To-Do Lists & Notes': 2, 'Event Finders': 2, 'Calendars & Reminders': 2, 'Wine & Beverages': 1, 'Restaurant Booking, Info & Reviews': 1, 'Game Info & Accessories': 1, 'Self Improvement': 1, 'Movie Info & Reviews': 1, 'Streaming Services': 1, 'Podcasts': 1, 'Social Networking': 1}) 
**Partial    
   Counter({'': 15, 'Games': 9, 'Home Services': 7, 'Public Transportation': 5, 'Streaming Services': 3, 'Organizers & Assistants': 3, 'Knowledge & Trivia': 2, 'Fashion & Style': 2, 'Music Info, Reviews & Recognition Services': 2, 'Navigation & Trip Planners'

**CATEGORIES 	   #Skills
**Broken     
   Counter({'Lifestyle': 36, 'Business & Finance': 30, 'Games & Trivia': 28, 'Health & Fitness': 28, 'Food & Drink': 25, 'News': 24, 'Weather': 19, 'Travel & Transportation': 18, 'Music & Audio': 17, 'Education & Reference': 16, 'Smart Home': 13, 'Productivity': 11, 'Shopping': 10, 'Utilities': 7, 'Social': 6, 'Organizers & Assistants': 5, 'Connected Car': 4, 'Home Services': 4, 'Schools': 3, 'Novelty & Humour': 3, 'Sports': 3, 'Navigation & Trip Planners': 2, 'Wine & Beverages': 1, 'Cooking & Recipes': 1, 'Restaurant Booking, Info & Reviews': 1, 'Event Finders': 1, 'To-Do Lists & Notes': 1, 'Movies & TV': 1, 'Podcasts': 1, '': 1, 'Calendars & Reminders': 1, 'Kids': 1, 'Novelty & Humor': 1}) 
**Partial    
   Counter({'Games & Trivia': 34, 'Lifestyle': 32, 'Health & Fitness': 25, 'Food & Drink': 20, 'Travel & Transportation': 18, 'Business & Finance': 16, 'Education & Reference': 16, 'Weather': 13, 'Shopping': 10, 'Productivity': 9, 'Social': 7, '

# Extra. Perform analysis of how many skills talk about certain websites

In [12]:
interesting_links = {'voiceskillsinc.com': set([]), 'topvoiceapps.com':set([]),'voxogenic.com':set([]) }
for key,value in allo.items():
    print('*market ', key)
    count = 0
    for skill in value:
        #print(skill)
        if('Skill_description' in skill):
            count+=1
            desc = skill['Skill_description'].lower()
            for iw in interesting_links.keys():
                if iw in desc:
                    interesting_links[iw].add(( skill['Name'], skill['Developer'], skill['Skill_description'] ) )
    print('processed ', count, 'skills with description')

print()
for k,v in interesting_links.items():
    print(k, ' ', len(v))
    
print()
print('****SKILLS with this word in description')
for k,v in interesting_links.items():
    print()
    print('*', k,'\n')
    for i in range(8):
        print('\t', list(v)[i])


*market  skills/US/file.json
processed  88419 skills with description
*market  skills/IN/file.json
processed  35005 skills with description
*market  skills/IT/file.json
processed  5269 skills with description
*market  skills/CA/file.json
processed  42429 skills with description
*market  skills/UK/file.json
processed  38705 skills with description
*market  skills/AU/file.json
processed  36794 skills with description
*market  skills/MX/file.json
processed  2304 skills with description
*market  skills/DE/file.json
processed  31879 skills with description
*market  skills/FR/file.json
processed  2562 skills with description
*market  skills/ES/file.json
processed  17588 skills with description
*market  skills/JP/file.json
processed  16054 skills with description

voiceskillsinc.com   9
topvoiceapps.com   15
voxogenic.com   42

****SKILLS with this word in description

* voiceskillsinc.com 

	 ('A Few Minutes In Voice', 'by VoiceSkillsInc', 'Description\nThis Podcast Flash Briefing provides u